<a href="https://colab.research.google.com/github/Sen-Ma/PUBG-Kaggle/blob/master/Hardest_way_to_get_a_t_shirt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf 
import re
import gc
import os
import datetime

kernel_start_time = datetime.datetime.now()

In [0]:
train = pd.read_csv('/content/gdrive/My Drive/pubg/train_V2.csv')
test = pd.read_csv('/content/gdrive/My Drive/pubg/test_V2.csv')

In [0]:
def reduce_mem_usage(df):
  for col in df.columns : 
    col_type = df[col].dtype

    if col_type != object:
      c_min = df[col].min()
      c_max = df[col].max()
      if str(col_type)[:3] == 'int':
        if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
          df[col] = df[col].astype(np.int8)
        elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
          df[col] = df[col].astype(np.int16)
        elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
          df[col] = df[col].astype(np.int32)
        elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
          df[col] = df[col].astype(np.int64)
      else : 
        if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
          df[col] = df[col].astype(np.float16)
        elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
          df[col] = df[col].astype(np.float32)
        else :
          df[col] = df[col].astype(np.float64)
    col_type = str(df[col].dtype)
    if col_type == 'object':
      col_type = 'str'
    else : 
      col_type = 'np.' + col_type
  return df

In [0]:
data_types = {'Id' : str, 'groupId':str, 'matchId' : str, 'assists' : np.int8, 'boosts' : np.int8, 'damageDealt' : np.float16, 'DBNOs' : np.int8, 'heatshotKills' : np.int8, 'heals' : np.int8, 'killPlace' : np.int8, 'killPoints' : np.int16, 'kills':np.int8, 'killStreaks':np.int8, 'longestKill' : np.float16, 'matchDuration' : np.int16, 'matchType' : str, 'maxPlace': np.int8, 'numGroups' : np.int8, 'rankPoints' : np.int16, 'revives' : np.int8, 'rideDistance' : np.float16, 'roadKills' : np.int8, 'swimDistance' : np.float16, 'teamKills': np.int8, 'vehicleDestorys' : np.int8, 'walkDistance' : np.float16, 'weaponsAcquired' : np.int16, 'winPoints':np.int16, 'winPlacePerc' : np.float16}

In [0]:
data_types['walkDistance'] = np.float32
data_types['rideDistance'] = np.float32
data_types['damageDealt'] = np.float32

In [0]:
zero_stdev_cols = ['median_match_roadKills', 'median_match_vehicleDestorys', 'median_match_road_kills_per_rideDistance', 'std_match_longestKill']
ulesess_stat_cols = ['median_match_revives', 'rmedian_match_teamKills', 'median_match_swimDistance_norm', 'max_match_kill_streak_rate','max_group_roadKills','min_group_roadKills','median_match_assists','sum_group_roadKills','sum_group_vechicleDestorys','min_group_vehicleDestorys']
useless_match_type_names = ['normal-duo','flarefpp','normal-solo','crashtpp','normal-squad','flaretpp','normal-duo-fpp','normal-squadfpp','normal-squad-fpp','crashfpp','duo','squad-fpp','squad-fpp','normal-solo-fpp']
all_useless_cols = zero_stdev_cols + ulesess_stat_cols + useless_match_type_names
all_useless_cols = set(all_useless_cols)
min_match_useful_cols = ['min_match_rankPoints', 'min_match_winPoints','min_match_killPoints']

In [0]:
def fix_missing_ranks(X, mean_ranks = None, rank_stds = None, rank_cols = ['rankPoints','winPoints']):
  if (mean_ranks is None) or (rank_stds is None):
    mean_ranks = {}
    rank_stds = {}

    for rank_col in rank_cols :
      mean_ranks[rank_col] = X.loc[X[rank_col] > 1, rank_col].mean()
      rank_stds[rank_col] = X.loc[X[rank_col] > 1, rank_col].std()

  rank_deltas = (X.loc[(X['rankPoints']>1)&(X['winPoints'] <= 1), 'rankPoints'] - mean_ranks['rankPoints']) / rank_stds['rankPoints']
  X.loc[(X['rankPoints'] > 1)&(X['winPoints'] <=1), 'winPoints'] = mean_ranks['winPoints'] + rank_stds['winPoints']*rank_deltas
  
  win_deltas = (X.loc[(X['rankPoints'] <= 1) & (X['winPoints'] > 1), 'winPoints'] - mean_ranks['winPoints']) / rank_stds['winPoints']
  X.loc[(X['rankPoints'] <= 1) & (X['winPoints']>1), 'rankPoints'] = mean_ranks['rankPoints'] + rank_stds['rankPoints']*win_deltas
  
  X.loc[(X['rankPoints'] <= 1) & (X['winPoints'] <= 1), 'rankPoints'] = mean_ranks['rankPoints']
  X.loc[(X['rankPoints'] <= 1) & (X['winPoints'] <= 1), 'winPoints'] = mean_ranks['winPoints']


  return X, mean_ranks, rank_stds

In [0]:
def add_player_features(X):
  X['headshot_rate'] = X['headshotKills'] / (X['kills'] + 0.00001)
  X['kill_streak_rate'] = X['killStreaks'] / (X['kills'] + 0.00001)
  X['kills_assists'] = X['kills'] + X['assists']
  X['heals_boosts'] = X['heals'] + X['boosts']
  X['total_distance'] = X['walkDistance'] + X['rideDistance'] + X['swimDistance']
  X['kills_assists_per_heal_boost'] = X['kills_assists'] / (X['heals_boosts'] + 1)
  X['road_kills_per_ride_Distance'] = X['roadKills'] / (X['rideDistance'] + 0.01)
  X['maxPlace_per_numGroups'] = X['maxPlace'] / X['numGroups']
  X['assists_per_kill'] = X['assists'] / (X['kills'] + X['assists'] + 0.00001)
  X['killPlace'] = X['killPlace'] - 1
  return X 

In [0]:
def create_basic_group_info(X):
  group_cols = ['matchId', 'groupId', 'matchDuration', 'matchType', 'maxPlace', 'numGroups','maxPlace_per_numGroups','winPlacePerc','killPlace']
  if 'winPlacePerc' not in X.columns:
    group_cols.remove('winPlacePerc')

  pl_data_grouped = X[group_cols].groupby(['matchId', 'groupId'])
  gr_data = pl_data_grouped.first()
  gr_data.drop(columns = 'killPlace', inplace = True)

  gr_data['raw_groupSize'] = pl_data_grouped['numGroups'].count()
  gr_data['groupSize'] = gr_data['raw_groupSize']
  gr_data['group_size_overflow'] = (gr_data['groupSize'] > 4).astype(np.int8)
  gr_data.loc[gr_data['groupSize'] > 4, ['groupSize']] = 2
  gr_data['meanGroupSize'] = gr_data.groupby('matchId')['groupSize'].transform(np.mean)
  gr_data['medianGroupSize'] = gr_data.groupby('matchId')['groupSize'].transform(np.median)
  gr_data['maxKillPlace'] = pl_data_grouped['killPlace'].max().groupby('matchId').transform(np.max)
  
  gr_data['totalPlayers'] = gr_data.groupby('matchId')['groupSize'].transform(sum)
  gr_data['totalPlayersAdjusted'] = gr_data['maxPlace'].astype(float)*gr_data['totalPlayers'] / gr_data['totalPlayersAdjusted'].apply(lambda x : np.minimum(100.0, x))
  gr_data['num_opponents'] = gr_data['totalPlayersAdjusted'] - gr_data['groupSize']

  X = X.merge(gr_data[['num_opponents', 'totalPlayersAdjusted', 'groupSize', 'raw_groupSize', 'maxKillPlace']], on= ['matchId', 'groupId'])

  print('group size counts : ')
  print(X['raw_groupSize'].value_counts())

  X['revives_per_groupSize'] = X['revives'] / (X['groupSize'] -1+0.001)
  X['kills_assists_norm_both'] = X['kills_assists'].astype(np.float32) / X['num_opponents'] / X['matchDuration']
  
  X['killPlace_norm'] = X['killPlace']/(X['maxKillPlace'] + 0.000001)

  X['damageDealt_norm_both'] = X['damageDealt'].astype(np.float32) / X['num_oppnents'] / X['matchDuration']
  X['DBNOs_norm'] = X['DBNOs'].astype(np.float32) / X['num_oppnents'] / X['matchDuration']
  X['heals_norm'] = X['heals'].astype(np.float32) / X['matchDuration']
  X['boosts_norm'] = X['boosts'].astype(np.float32) / X['matchDuration']
  X['walkDistance_norm'] = X['walkDistance'].astype(np.float32) / X['matchDuration']
  X['rideDistance_norm'] = X['rideDistance'].astype(np.float32) / X['matchDurtaion']
  X['swimDistance_norm'] = X['swimDistance'].astype(np.float32) / X['matchDurtaion']

  